##**🗂️ ¿Qué es una base de datos?**  
Una base de datos es un sistema organizado para almacenar, gestionar y acceder a grandes cantidades de información de forma eficiente. Está diseñada para que los datos puedan ser consultados, modificados y mantenidos con facilidad, ya sea por aplicaciones, usuarios o procesos automatizados. Las bases de datos son fundamentales en casi todos los sistemas digitales, desde redes sociales hasta tiendas en línea.

##**🔗 ¿Qué es una base de datos relacional?**  
Una base de datos relacional organiza la información en tablas, donde cada fila representa un registro y cada columna un atributo. Estas tablas se relacionan entre sí mediante claves (primarias y foráneas), lo que permite estructurar los datos y evitar duplicidades. Este modelo facilita consultas complejas, integridad de los datos y escalabilidad, siendo el más utilizado en sistemas empresariales.

##**💻 ¿Qué es SQL y por qué es importante?**  
SQL (Structured Query Language) es el lenguaje estándar para interactuar con bases de datos relacionales. Permite consultar, insertar, actualizar y eliminar datos, así como definir estructuras y relaciones. Es una herramienta esencial para analistas, desarrolladores y científicos de datos, ya que brinda el control necesario para explorar, transformar y analizar grandes volúmenes de información de forma precisa y eficiente.


## 🧬 Radiografía de una consulta SQL

Una consulta SQL sigue una estructura lógica compuesta por bloques que se ejecutan en un orden específico, aunque no necesariamente en el orden en que los escribimos. Aquí tienes un esquema con los principales componentes:

```
SELECT         -- ¿Qué columnas quiero ver?
  columnas_o_expresiones
FROM           -- ¿De qué tabla(s)?
  tabla_base_o_joins
[WHERE]        -- ¿Qué filas quiero filtrar?
  condiciones_de_filtro
[GROUP BY]     -- ¿Cómo agrupo los datos?
  columna_agrupadora
[HAVING]       -- ¿Qué grupos cumplen una condición?
  condiciones_sobre_agregados
[ORDER BY]     -- ¿En qué orden quiero los resultados?
  columna [ASC|DESC]
[LIMIT]        -- ¿Cuántas filas mostrar?
  número_de_filas
```

---

### 🔍 Ejemplo comentado

```sql
SELECT first_name, last_name, COUNT(*) AS cantidad_rentas  -- Qué columnas mostrar
FROM customer c                                            -- De qué tabla
JOIN rental r ON c.customer_id = r.customer_id             -- Unir con tabla relacionada
WHERE c.active = 1                                         -- Filtrar clientes activos
GROUP BY c.customer_id                                     -- Agrupar por cliente
HAVING COUNT(*) > 10                                       -- Solo clientes con más de 10 rentas
ORDER BY cantidad_rentas DESC                              -- Ordenar por cantidad descendente
LIMIT 5;                                                   -- Solo los 5 primeros
```

---

### 🧠 Nota importante
El orden de ejecución **lógico** en SQL es:

1. `FROM` (+ `JOIN`)
2. `WHERE`
3. `GROUP BY`
4. `HAVING`
5. `SELECT`
6. `ORDER BY`
7. `LIMIT`

Aunque escribas `SELECT` al inicio, ¡SQL primero arma las tablas y filtra filas antes de mostrar columnas!


## Cargar base de datos

In [1]:
# Paso 1: Clonar el repositorio
!git clone https://github.com/bradleygrant/sakila-sqlite3.git

Cloning into 'sakila-sqlite3'...
remote: Enumerating objects: 18, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 18 (delta 2), reused 1 (delta 1), pack-reused 14 (from 1)
Receiving objects: 100% (18/18), 2.39 MiB | 2.52 MiB/s, done.
Resolving deltas: 100% (4/4), done.


In [2]:
import sqlite3
import pandas as pd

In [3]:
# Crear conexión
conn = sqlite3.connect("/content/sakila-sqlite3/sakila_master.db")

In [4]:
# Ver tablas disponibles
query = "SELECT name FROM sqlite_master WHERE type='table';"
tables = pd.read_sql(query, conn)
print("Tablas en la base de datos:")
print(tables)


Tablas en la base de datos:
             name
0           actor
1         country
2            city
3         address
4        language
5        category
6        customer
7            film
8      film_actor
9   film_category
10      film_text
11      inventory
12          staff
13          store
14        payment
15         rental


In [8]:
conn.close()

In [9]:
def run_query(query):
  conn = sqlite3.connect("/content/sakila-sqlite3/sakila_master.db")
  df = pd.read_sql(query, conn)
  conn.close()
  return df

## 🧪 Ejercicios de SQL con la base Sakila (SQLite)

Trabajaremos sobre la base de datos `sakila.db` montada en el entorno. Asegúrate de tener activa tu conexión con SQLite. A continuación, encontrarás 20 ejercicios ordenados por nivel de complejidad para practicar distintos aspectos del lenguaje SQL.

---


### 1️⃣ Selección básica  
**Consulta:** Muestra los primeros 10 actores con su `first_name` y `last_name`.

```sql
SELECT first_name, last_name
FROM actor
LIMIT 10;
```

---

### 2️⃣ Filtrado con condiciones  
**Consulta:** Encuentra todas las películas con una duración mayor a 180 minutos.

```sql
SELECT title, length
FROM film
WHERE length > 180;
```

---


### 3️⃣ Operadores lógicos  
**Consulta:** Muestra las películas con clasificación 'PG' o 'PG-13'.

```sql
SELECT title, rating
FROM film
WHERE rating IN ('PG', 'PG-13');
```

---

### 4️⃣ Ordenamiento  
**Consulta:** Lista los clientes ordenados por `last_name` ascendente y `first_name` descendente.

```sql
SELECT first_name, last_name
FROM customer
ORDER BY last_name ASC, first_name DESC;
```

---



### 5️⃣ LIKE y patrones  
**Consulta:** Encuentra todas las películas cuyo título comienza con "A".

```sql
SELECT title
FROM film
WHERE title LIKE 'A%';
```

---

### 6️⃣ Agrupación por categoría  
**Consulta:** Cuenta cuántas películas hay por tipo de clasificación (`rating`).

```sql
SELECT rating, COUNT(*) AS total
FROM film
GROUP BY rating;
```

### 7️⃣ HAVING y filtros de agregados  
**Consulta:** Muestra solo las clasificaciones que tienen más de 200 películas.

```sql
SELECT rating, COUNT(*) AS total
FROM film
GROUP BY rating
HAVING total > 200;
```

---

### 8️⃣ INNER JOIN  
**Consulta:** Muestra los títulos de las películas y los nombres de los actores que actúan en ellas.

```sql
SELECT f.title, a.first_name || ' ' || a.last_name AS actor_name
FROM film f
JOIN film_actor fa ON f.film_id = fa.film_id
JOIN actor a ON a.actor_id = fa.actor_id
LIMIT 10;
```

---


### 9️⃣ LEFT JOIN  
**Consulta:** Muestra todos los clientes y sus pagos (si existen).

```sql
SELECT c.first_name || ' ' || c.last_name AS cliente, p.amount
FROM customer c
LEFT JOIN payment p ON c.customer_id = p.customer_id
LIMIT 10;
```

---


### 🔟 Agrupación con SUM  
**Consulta:** Muestra cuánto ha pagado en total cada cliente.

```sql
SELECT c.first_name || ' ' || c.last_name AS cliente, SUM(p.amount) AS total_pagado
FROM customer c
JOIN payment p ON c.customer_id = p.customer_id
GROUP BY c.customer_id;
```

---

### 1️⃣1️⃣ Subconsulta en WHERE  
**Consulta:** Muestra películas más largas que el promedio general de duración.

```sql
SELECT title, length
FROM film
WHERE length > (
    SELECT AVG(length) FROM film
);
```

---


### 1️⃣2️⃣ JOIN de múltiples tablas  
**Consulta:** Muestra el nombre del cliente y la película que rentó.

```sql
SELECT c.first_name || ' ' || c.last_name AS cliente, f.title
FROM customer c
JOIN rental r ON c.customer_id = r.customer_id
JOIN inventory i ON r.inventory_id = i.inventory_id
JOIN film f ON i.film_id = f.film_id
LIMIT 10;
```

---

### 1️⃣3️⃣ Funciones de fecha  
**Consulta:** Cuenta cuántos pagos se realizaron por mes.

```sql
SELECT strftime('%Y-%m', payment_date) AS mes, COUNT(*) AS pagos
FROM payment
GROUP BY mes
ORDER BY mes;
```

---


### 1️⃣4️⃣ CASE para categorizar  
**Consulta:** Categoriza las películas según su duración.

```sql
SELECT title, length,
  CASE
    WHEN length < 60 THEN 'Corta'
    WHEN length BETWEEN 60 AND 120 THEN 'Media'
    ELSE 'Larga'
  END AS categoria
FROM film
LIMIT 10;
```

---


### 1️⃣5️⃣ COUNT con DISTINCT  
**Consulta:** ¿Cuántos clientes únicos han realizado al menos un pago?

```sql
SELECT COUNT(DISTINCT customer_id) AS clientes_que_pagaron
FROM payment;
```

---

### 1️⃣6️⃣ Tabla derivada (subconsulta en FROM)  
**Consulta:** ¿Cuál es el promedio de duración por clasificación?

```sql
SELECT rating, avg_length
FROM (
  SELECT rating, AVG(length) AS avg_length
  FROM film
  GROUP BY rating
);
```


In [16]:
query="""
SELECT rating, avg_length
FROM (
  SELECT rating, AVG(length) AS avg_length
  FROM film
  GROUP BY rating
);
"""

run_query(query)

,rating,avg_length
0,G,111.050562
1,NC-17,113.228571
2,PG,112.005155
3,PG-13,120.443946
4,R,118.661538



### 1️⃣7️⃣ Función de ventana: RANK  
**Consulta:** Muestra el top 10 de clientes que más han pagado usando ranking.

```sql
SELECT first_name || ' ' || last_name AS cliente,
       SUM(amount) AS total_pagado,
       RANK() OVER (ORDER BY SUM(amount) DESC) AS ranking
FROM customer
JOIN payment ON customer.customer_id = payment.customer_id
GROUP BY customer.customer_id
LIMIT 10;
```

### 1️⃣8️⃣ Función de ventana: PARTITION BY  
**Consulta:** Muestra el total pagado por cada cliente y la suma total por tienda.

```sql
SELECT store_id, first_name || ' ' || last_name AS cliente,
       SUM(amount) AS total_cliente,
       SUM(SUM(amount)) OVER (PARTITION BY store_id) AS total_por_tienda
FROM customer
JOIN payment ON customer.customer_id = payment.customer_id
GROUP BY customer.customer_id;
```


### 1️⃣9️⃣ ROW_NUMBER()  
**Consulta:** Asigna un número de fila a cada película ordenada por duración descendente.

```sql
SELECT title, length,
       ROW_NUMBER() OVER (ORDER BY length DESC) AS fila
FROM film
LIMIT 10;
```



### 2️⃣0️⃣ LAG y comparación entre pagos  
**Consulta:** Muestra el monto de pago de cada cliente y su diferencia con el pago anterior.

```sql
SELECT customer_id, amount,
       LAG(amount, 1) OVER (PARTITION BY customer_id ORDER BY payment_date) AS pago_anterior
FROM payment
ORDER BY customer_id, payment_date
LIMIT 20;
```
